### Análisis de Horas Críticas del Mercado para el Balance de Potencia ###
**Author:** Aminadab Cordova

**Date:** 2024-05-02

In [1]:
# Módulos requeridos
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Conexión a Base de Datos

In [3]:
# Consulta SQL para las horas criticas del año de producción
sql_query = """SELECT * FROM public."Horas_Criticas_AP" """

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_hc_ap = pd.read_sql(sql_query, cnx_mbp)

In [5]:
# Se crea el dataframe con horas criticas definitivas del SIN
df_hc_ap_sin = df_hc_ap[df_hc_ap["Zona_Potencia"] == "SIN"]
df_hc_ap_sin = df_hc_ap_sin[df_hc_ap_sin["Tipo_Dato"] == "Definitivo"]

# se forma dataframe solo con fecha y hora
df_hc = df_hc_ap_sin[['Fecha', 'Hora']].reset_index(drop=True)
df_hc['Fecha'] = pd.to_datetime(df_hc['Fecha'])
df_hc

,Fecha,Hora
0,2016-05-23,16
1,2016-05-23,17
2,2016-05-24,15
3,2016-05-24,16
4,2016-05-24,17
...,...,...
795,2023-09-28,20
796,2023-09-28,21
797,2023-09-28,22
798,2023-10-05,19


In [12]:
# Se define el rango de fechas de los años de Produccion
date_range = pd.date_range('2016-01-01', '2023-12-31')

# Se repite cada día para formar una serie horaria
fecha = np.repeat(date_range, 24)

# se crea dataframe conla serie de fechas
df_serie = pd.DataFrame (fecha, columns=['Fecha'])

# Se define un arreglo de las horas y se agrega a df_serie
hour=np.arange(start=1, stop=25, step=1)
df_serie['Hora'] = np.tile(hour, len(date_range))

df_serie

,Fecha,Hora
0,2016-01-01,1
1,2016-01-01,2
2,2016-01-01,3
3,2016-01-01,4
4,2016-01-01,5
...,...,...
70123,2023-12-31,20
70124,2023-12-31,21
70125,2023-12-31,22
70126,2023-12-31,23


In [13]:
# Realizar una combinación externa (outer join) en los dos DataFrames
df_merged = df_serie.merge(df_hc, on=['Fecha', 'Hora'], how='outer', indicator=True)

# Agregar una columna llamada 'HC' que indique si la hora es crítica o no
df_merged['HC'] = df_merged['_merge'] == 'both'

# Eliminar la columna de indicador '_merge'
df_merged = df_merged.drop('_merge', axis=1)

In [16]:
df_merged[df_merged['HC'] == True]

,Fecha,Hora,HC
1623,2016-03-08,16,True
1624,2016-03-08,17,True
2367,2016-04-08,16,True
2368,2016-04-08,17,True
2369,2016-04-08,18,True
...,...,...,...
67867,2023-09-28,20,True
67868,2023-09-28,21,True
67869,2023-09-28,22,True
68034,2023-10-05,19,True
